# Extração de grafos de conhecimento de bulas de medicamentos de arquivos PDF extraídos do bulário eletrônico da ANVISA


**Materiais úteis**
- [Preparing PDFs for RAGs: um comparativo entre bibliotecas](https://towardsdatascience.com/preparing-pdfs-for-rags-b1579fc697f1/)
- [Build a knowledge graph from documents using Docling](https://alain-airom.medium.com/build-a-knowledge-graph-from-documents-using-docling-8bc05e1389f7)
- [Comprehensive Guide to Chunking in LLM and RAG Systems](https://kshitijkutumbe.medium.com/comprehensive-guide-to-chunking-in-llm-and-rag-systems-c579a11ce6e2)
- [Building Knowledge Graphs with LLM Graph Transformer](https://medium.com/data-science/building-knowledge-graphs-with-llm-graph-transformer-a91045c49b59)
- [How to construct knowledge graphs](https://python.langchain.com/docs/how_to/graph_constructing/)
- [Geração com Recuperação Aumentada (RAG) em
Grafos de Conhecimento](https://books-sol.sbc.org.br/index.php/sbc/catalog/download/153/658/1179?inline=1)

In [18]:
# Rodar somente quando estiver fazendo a preparaçaõ do ambiente de desenvolvimento para fazer o download da biblioteca
# !python3 -m spacy download pt_core_news_md

In [19]:
from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from src.data_processing.pdf_and_markdown_pipeline import PdfAndMarkdownPipeline
from src.data_processing.chunks import HEADERS_TO_SPLIT_ON

from src.connetion.gen_ai_connection import LLMModel
from src.kg.kg_from_text import KGFromText


FILE_NAME = "bula_1748876242121.pdf"
PDF_FILE_PATH = f"{RAW_DATA_DIR}/{FILE_NAME}"
MD_FILE_PATH = f"{PROCESSED_DATA_DIR}/{FILE_NAME}.md"


chunk_size = 400
chunk_overlap = 100
headers_to_split_on = HEADERS_TO_SPLIT_ON

provider = "openai"
temperature = 0.7
max_tokens = None

figsize = (10, 8)
show_node_properties = False

A classe PdfAndMarkdownPipeline possui as seguintes funcionalidades:
- Converte um arquivo pdf para markdown
- Carrega o conteúdo de um arquivo markdown e gera os chunks do arquivo
- Os chunks são gerados em duas etapas: a primeira divisão se dá pelo título das seções e depois divide o conteúdo das seções de acordo com o tamanho dos chunks configurados

In [20]:
pdf_md_pipeline = PdfAndMarkdownPipeline()

In [21]:
# Executar somente quando for converter um pdf para markdown
# pdf_md_pipeline.pdf_to_markdown(
#     source_file_path=PDF_FILE_PATH,
#     dest_dir_path=PROCESSED_DATA_DIR
# )

In [22]:
# Executar sempre que for ler o conteúdo do markdown
pdf_md_pipeline.load_markdown(
    source_file_path=MD_FILE_PATH
)
pdf_md_pipeline.md_documents

INFO:src.data_processing.pdf_and_markdown_pipeline:Document /home/marilia/git/pos/tcc_ufg_akcit_nlp/data/processed/bula_1748876242121.pdf.md loaded.


[Document(metadata={'source': '/home/marilia/git/pos/tcc_ufg_akcit_nlp/data/processed/bula_1748876242121.pdf.md'}, page_content="paracetamol\n\nAché Laboratórios Farmacêuticos S.A. C omprimidos revestidos 750 mg\n\nBULA PARA PACIENTE\n\nBula de acordo com a Resolução-RDC nº 47/2009\n\nI.IDENTIFICAÇÃO DO MEDICAMENTO\n\nparacetamol\n\nMedicamento Genérico Lei nº 9.787, de 1999\n\nAPRESENTAÇÕES\n\nComprimidos revestidos de 750 mg: embalagens com 20 ou 200 comprimidos.\n\nUSO ORAL\n\nUSO ADULTO E PEDIÁTRICO ACIMA DE 12 ANOS\n\nCOMPOSIÇÃO\n\nCada comprimido revestido contém:\n\nparacetamol..........................................................................................................................................750 mg\n\nExcipientes: povidona, amidoglicolato de sódio, amido, ácido esteárico, álcool polivinílico, macrogol, talco e dióxido de titânio.\n\nII. INFORMAÇÕES AO PACIENTE\n\n1. PARA QUE ESTE MEDICAMENTO É INDICADO?\n\nEste medicamento é indicado para a redução da febre 

In [23]:
pdf_md_pipeline.get_chunks(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    headers_to_split_on=headers_to_split_on,
)
pdf_md_pipeline.chunk_documents

[Document(metadata={}, page_content='paracetamol  \nAché Laboratórios Farmacêuticos S.A. C omprimidos revestidos 750 mg  \nBULA PARA PACIENTE  \nBula de acordo com a Resolução-RDC nº 47/2009  \nI.IDENTIFICAÇÃO DO MEDICAMENTO  \nparacetamol  \nMedicamento Genérico Lei nº 9.787, de 1999  \nAPRESENTAÇÕES  \nComprimidos revestidos de 750 mg: embalagens com 20 ou 200 comprimidos.  \nUSO ORAL  \nUSO ADULTO E PEDIÁTRICO ACIMA DE 12 ANOS'),
 Document(metadata={}, page_content='USO ORAL  \nUSO ADULTO E PEDIÁTRICO ACIMA DE 12 ANOS  \nCOMPOSIÇÃO  \nCada comprimido revestido contém:  \nparacetamol..........................................................................................................................................750 mg  \nExcipientes: povidona, amidoglicolato de sódio, amido, ácido esteárico, álcool polivinílico, macrogol, talco e dióxido de titânio.'),
 Document(metadata={}, page_content='II. INFORMAÇÕES AO PACIENTE  \n1. PARA QUE ESTE MEDICAMENTO É INDICADO?  \nEste medicamen

A classe LLMModel carrega o modelo LLM de acordo com o provedor de modelo configurado que pode ser:
- local
- openai
- google
- groq
- hf (para huggingface)

In [24]:
llm = LLMModel(
    provider=provider,
    temperature=temperature,
    max_tokens=max_tokens
).llm

In [25]:
pdf_md_pipeline.chunk_documents[5]

Document(metadata={}, page_content='4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO?  \nVocê não deve tomar mais do que a dose recomendada (superdose) para provocar maior alívio, pois pode causar sérios problemas de saúde. Você deve consultar seu médico se a dor ou febre continuar ou piorar, ou se surgirem novos sintomas, pois estes sintomas podem ser sinais de doenças graves.')

A classe KGFromText gera o grafo de conhecimento a partir de um modelo LLM  utilizando o LLMGraphTransformer do langchain.

In [26]:
kg = KGFromText(
    llm=llm
)

In [27]:
chunk_documents = [pdf_md_pipeline.chunk_documents[5]]

await kg.get_kg(
    chunk_documents=chunk_documents
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


KGFromText(llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x75424d50ab40>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x75424d509a30>, root_client=<openai.OpenAI object at 0x75424d569340>, root_async_client=<openai.AsyncOpenAI object at 0x75424d568140>, model_name='gpt-4o-mini', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********')), graph_documents=[GraphDocument(nodes=[Node(id='Medicament', type='Concept', properties={}), Node(id='Dose Recomendada', type='Concept', properties={}), Node(id='Problemas De Saúde', type='Concept', properties={}), Node(id='Médico', type='Person', properties={}), Node(id='Dor', type='Symptom', properties={}), Node(id='Febre', type='Symptom', properties={}), Node(id='Sintomas Novos', type='Symptom', properties={}), Node(id='Doenças Graves', type='Concept', properties={})], relationships=[Relationship(source=Node(id='Medicament', type='Concept', properti

In [28]:
kg.graph_documents

[GraphDocument(nodes=[Node(id='Medicament', type='Concept', properties={}), Node(id='Dose Recomendada', type='Concept', properties={}), Node(id='Problemas De Saúde', type='Concept', properties={}), Node(id='Médico', type='Person', properties={}), Node(id='Dor', type='Symptom', properties={}), Node(id='Febre', type='Symptom', properties={}), Node(id='Sintomas Novos', type='Symptom', properties={}), Node(id='Doenças Graves', type='Concept', properties={})], relationships=[Relationship(source=Node(id='Medicament', type='Concept', properties={}), target=Node(id='Dose Recomendada', type='Concept', properties={}), type='DEVE_SEGUIR', properties={}), Relationship(source=Node(id='Medicament', type='Concept', properties={}), target=Node(id='Problemas De Saúde', type='Concept', properties={}), type='PODE_CAUSAR', properties={}), Relationship(source=Node(id='Dor', type='Symptom', properties={}), target=Node(id='Médico', type='Person', properties={}), type='CONSULTAR', properties={}), Relationship

In [ ]:
kg.plot_and_save_graph_visualization(
    file_name=FILE_NAME,
    figsize=figsize, 
    show_node_properties=show_node_properties
)

AttributeError: 'KGFromText' object has no attribute 'plot_and_save_graph_visualization'

In [ ]:
kg.save_graph_on_db()